<font size="3"> Проект <font size="4"> "Активность пользователя в ЛК - анализ оттока"

_Описание_

**Проверяем 2 гипотезы:**

активный пользователь более опытен, знает как получить желаемое, не склонен быстро уходить с платформы

активный пользователь быстрее понимает, что результаты плохие - и уходит

**Группы по признакам:**

частота захода в ЛК (по логам авторизации) - исключаем админов и авторизацию с офисного IP

количество используемых форматов. Разделяем popunder, banner, in-stream, in-page, web-push и other (в other относим native ad, gallery, notification widget и прочую мелочь. ТМА в отдельный формат не выделяем

количество созданных кампаний

количество редактирований кампании (любых) - по логам, редактирование только юзером

количество редактирований ставки (price) - аналогично. Автоаджаст не в счет. Если в какой-то когорте много юзеров на автоаджасте - возможно стоит их выделить в отдельную группу.

количество редактирований таргетов (target) - аналогично

Все действия (заход в ЛК, создание кампании или редактирование) должны происходить именно в тот период, когда мы присвоили пользователю метку - churned / not churned, то есть по сути совпадать с границами когорты.
Пример: когорта июня (01.06.2025) - для кадого пользователя этой когорты нам нужно посчитать сколько кампаний он создал за период с 01.06.2025 по 30.06.2025 включительно. Все, что происходило за рамками этого периода - нам не интересно.
Для задачи с редактированием кампаний, цен и таргетов - нас интересуют действия пользователя, произошедшие в границах когорты. Ограничения по дате создания кампании нет. Здесь количество когорт ограничится теми месяцами, которые попали в логи. По сути мы можем проанализировать только когорту мая, июня и июля.

Показатели считаем поюзерно. Далее вычисляем среднее значение по группе. 

Группы формируем по корзинам.
Пример групп по количеству созданных капманий:

1 кампания 

2 - 5 кампаний

6 - 20 кампаний

21+ кампаний (скорее всего работает через API)

Размер корзин аналитик определяет экспертно исходя из распределения данных.

_Итоговая цель: найти сегменты (группы, подгруппы), в которых доля оттока выше - там будем в последующем искать причины._

**Методика работы:**

Делим клиентов на группы

Считаем размер групп в юзерах и в %.

Считаем долю оттока в каждой группе.

Выделяем группы, которые кажутся нам аномальными, описываем, делаем выводы

#### Подключение к БД и импорты библиотек

In [2]:
import os
import pandas as pd
import numpy as np
import sqlalchemy as sql
from sqlalchemy import create_engine

In [3]:
db_config = {
'host': 'host',
'port' : 3306,
'db' : 'db',
'user' : os.getenv('MYSQL_USER'),
'pass' : os.getenv('MYSQL_PASSWORD'),
}
 
# connection_string = "mysql+pymysql://<dbuser>:<pwd>@<server_ip>/<db_instance>"
connection_string = 'mysql+pymysql://{}:{}@{}/{}'.format(
    db_config['user'],
    db_config['pass'],
    db_config['host'],
    db_config['db'],
)

In [4]:
engine = create_engine(connection_string)

In [5]:
def select(sql):
  return pd.read_sql(sql, con=engine)

#### SQL-запросы

In [73]:
sql = '''
WITH first_spend AS (
  SELECT 
    payments.user_id,
    DATE_FORMAT(DATE_ADD(MIN(payments.created_at), INTERVAL 1 MONTH), '%%Y-%%m-01') AS cohort_month
  FROM payments
  LEFT JOIN users u ON payments.user_id = u.id
  WHERE payments.type = 'campaign'
    AND DATE(u.created_at) BETWEEN %(start_date)s AND %(end_date)s
  GROUP BY payments.user_id
  HAVING DATE_FORMAT(DATE_ADD(MIN(payments.created_at), INTERVAL 1 MONTH), '%%Y-%%m-01') <= DATE_FORMAT(CURDATE(), '%%Y-%%m-01')
),

cohort_checkpoints AS (
  SELECT 
    user_id,
    cohort_month,
    cohort_month AS check_month_start,
    LAST_DAY(cohort_month) AS check_month_end
  FROM first_spend
),

recent_spend AS (
  SELECT 
    cc.user_id,
    cc.cohort_month,
    cc.check_month_end,
    MAX(p.created_at) AS last_spend
  FROM cohort_checkpoints cc
  LEFT JOIN payments p
    ON p.user_id = cc.user_id
   AND p.type = 'campaign'
   AND p.created_at BETWEEN cc.check_month_start AND cc.check_month_end
  GROUP BY cc.user_id, cc.cohort_month, cc.check_month_end
),

classified AS (
  SELECT 
    cohort_month,
    user_id,
    CASE 
      WHEN last_spend >= DATE_SUB(check_month_end, INTERVAL 14 DAY) THEN 0
      ELSE 1
    END AS is_churned
  FROM recent_spend
),

first_spend_true AS (
  SELECT 
    payments.user_id,
    MIN(payments.created_at) AS first_spend
  FROM payments
  LEFT JOIN users u ON payments.user_id = u.id
  WHERE payments.type = 'campaign'
    AND DATE(u.created_at) BETWEEN '2024-12-01' AND '2025-06-30'
  GROUP BY payments.user_id
),

logins_window AS (
  SELECT 
    al.user_id,
    COUNT(*) AS cnt_logins
  FROM auth_logins al
  JOIN first_spend_true fs ON al.user_id = fs.user_id
  JOIN first_spend fsp ON al.user_id = fsp.user_id
  WHERE al.user_type LIKE '%%User%%'
    AND al.created_at <= LAST_DAY(fsp.cohort_month)
    AND al.ip_address NOT IN ('52.18.41.245', '54.194.2.114', '99.80.49.98', '109.206.170.203')
  GROUP BY al.user_id
),

camps_window AS (
  SELECT 
    c.user_id,
    COUNT(DISTINCT c.ad_format) AS cnt_formats,
    COUNT(*) AS cnt_camps_created
  FROM campaigns c
  JOIN first_spend_true fs ON c.user_id = fs.user_id
  JOIN first_spend fsp ON c.user_id = fsp.user_id
  WHERE c.created_at <= LAST_DAY(fsp.cohort_month)
  GROUP BY c.user_id
)

SELECT 
  classified.user_id,
  classified.cohort_month,
  classified.is_churned,
  COALESCE(logins_window.cnt_logins, 0) AS cnt_logins,
  camps_window.cnt_formats,
  camps_window.cnt_camps_created,
  camps_window.cnt_camps_created/camps_window.cnt_formats AS avg_camps_on_format
FROM classified
LEFT JOIN users u ON classified.user_id = u.id
LEFT JOIN logins_window ON classified.user_id = logins_window.user_id
LEFT JOIN camps_window ON classified.user_id = camps_window.user_id
WHERE u.email NOT LIKE '%%onlinesup%%'
'''

In [17]:
params = {
    'start_date': '2024-12-01',
    'end_date': '2025-06-30'
}

In [74]:
df = pd.read_sql(sql, con=engine, params=params)

In [40]:
df.head()

,user_id,cohort_month,is_churned,cnt_logins,cnt_formats,cnt_camps_created
0,173476,2025-03-01,1,8,1,1
1,173335,2025-04-01,1,26,1,1
2,169627,2025-02-01,1,24,2,4
3,170711,2025-03-01,1,2,1,4
4,180385,2025-05-01,0,50,1,5


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1651 entries, 0 to 1650
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            1651 non-null   int64 
 1   cohort_month       1651 non-null   object
 2   is_churned         1651 non-null   int64 
 3   cnt_logins         1651 non-null   int64 
 4   cnt_formats        1651 non-null   int64 
 5   cnt_camps_created  1651 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 77.5+ KB


#### Анализ групп

##### Распределение групп

Распределение данных для установления групп

In [94]:
df.describe()

,user_id,is_churned,cnt_logins,cnt_formats,cnt_camps_created,avg_camps_on_format
count,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000
mean,173991.357965,0.774682,22.456693,1.734706,12.067838,7.416290
std,10354.884410,0.417918,32.936133,1.089287,79.000600,40.974131
min,157310.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,164603.000000,1.000000,6.000000,1.000000,2.000000,1.250000
50%,173417.000000,1.000000,13.000000,1.000000,4.000000,2.000000
75%,182902.500000,1.000000,26.000000,2.000000,8.000000,5.000000
max,193389.000000,1.000000,551.000000,8.000000,2232.000000,1116.000000


In [99]:
df.query('cnt_logins == 0')

,user_id,cohort_month,is_churned,cnt_logins,cnt_formats,cnt_camps_created,avg_camps_on_format,camps_on_format_group
546,167478,2025-02-01,0,0,1,2,2.0,2
608,170935,2025-03-01,0,0,1,4,4.0,4


##### Отток по группам, разбитым по кол-ву авторизаций

Функция для присвоения группы по кол-ву авторизаций

In [102]:
def get_group(cnt):
        if cnt == 0:
            return '0'
        elif 1 <= cnt <= 6:
            return '1-6'
        elif 7 <= cnt <= 13:
            return '7-13'
        elif 14 <= cnt <= 26:
            return '14-26'
        else:
            return '>26'

Группы разбиты по квантилям : до 25% данных, 25-50, 50-75, и 75-100%

В финальной таблице убираем группу, где 0 авторизаций - в ней только два наблюдения

In [103]:
df['login_group'] = df['cnt_logins'].apply(get_group)

In [104]:
print('Отток по кол-ву логов')
df['login_group'].value_counts().reset_index().merge(round(df['login_group'].value_counts(normalize=True) * 100, 2),
on='login_group', how='left').merge(round(df.groupby('login_group')['is_churned'].mean() * 100, 2).sort_values(ascending=False).reset_index(),
                             on='login_group', how='left').sort_values(by='is_churned', ascending=False)[:4]

Отток по кол-ву логов


,login_group,count,proportion,is_churned
0,1-6,444,26.89,93.92
2,7-13,404,24.47,87.62
1,14-26,413,25.02,73.12
3,>26,388,23.50,53.09


Самый высокий отток в первых двух группах

##### Отток по группам, разбитым по кол-ву используемых форматов

In [105]:
print('Распределение данных по кол-ву используемых форматов')
df['cnt_formats'].value_counts()

Распределение данных по кол-ву используемых форматов


cnt_formats
1    950
2    380
3    202
4     75
5     27
6     10
8      4
7      3
Name: count, dtype: int64

Видим что в основном используют от 1 до 3 форматов. Всё, что больше - объединяем в одну группу

In [70]:
def get_group_formats(cnt):
        if cnt == 1:
            return '1'
        elif cnt == 2:
            return '2'
        elif cnt == 3:
            return '3'
        else:
            return '>= 4'

In [71]:
df['format_group'] = df['cnt_formats'].apply(get_group_formats)

In [72]:
print('Отток по кол-ву логов')
df['format_group'].value_counts().reset_index().merge(round(df['format_group'].value_counts(normalize=True) * 100, 2),
on='format_group', how='left').merge(round(df.groupby('format_group')['is_churned'].mean() * 100, 2).sort_values(ascending=False).reset_index(),
                             on='format_group', how='left').sort_values(by='is_churned', ascending=False)

Отток по кол-ву логов


,format_group,count,proportion,is_churned
0,1,950,57.54,83.89
1,2,380,23.02,72.37
3,>= 4,119,7.21,68.07
2,3,202,12.24,62.38


Самый высокий отток в первых двух группах

##### Отток по группам, разбитым по кол-ву созданных кампаний

In [61]:
df['cnt_camps_created'].describe()

count    1651.000000
mean       12.067838
std        79.000600
min         1.000000
25%         2.000000
50%         4.000000
75%         8.000000
max      2232.000000
Name: cnt_camps_created, dtype: float64

In [62]:
def get_group_camps(cnt):
        if cnt == 1:
            return '1'
        elif cnt == 2:
            return '2'
        elif 3 <= cnt <= 4:
            return '3-4'
        elif 5 <= cnt <= 8:
            return '5-8'
        else:
            return '>8'

Так же логичным кажется деление на группы по квантилям

In [63]:
df['camps_group'] = df['cnt_camps_created'].apply(get_group_camps)

In [66]:
df['camps_group'].value_counts().reset_index().merge(round(df['camps_group'].value_counts(normalize=True) * 100, 2),
on='camps_group', how='left').merge(round(df.groupby('camps_group')['is_churned'].mean() * 100, 2).sort_values(ascending=False).reset_index(),
                             on='camps_group', how='left').sort_values(by='is_churned', ascending=False)

,camps_group,count,proportion,is_churned
3,1,284,17.20,92.25
4,2,267,16.17,89.51
0,3-4,383,23.20,82.77
2,5-8,352,21.32,73.58
1,>8,365,22.11,55.34


Тоже видим что склонны к оттоку чаще пользователи, у которых не так много кампаний

##### Отток по группам, разбитым по кол-ву кампаний на один формат

Так же поступил запрос на изучение оттока в группах по использованию кампаний на одном формате

В данном случае мы просто делим кол-во кампаний юзера на кол-во используемых им форматов, и получаем кол-во кампаний, приходящих на один формат

In [75]:
df['avg_camps_on_format'].describe()

count    1651.000000
mean        7.416290
std        40.974131
min         1.000000
25%         1.250000
50%         2.000000
75%         5.000000
max      1116.000000
Name: avg_camps_on_format, dtype: float64

In [106]:
def get_group_camps_on_format(cnt):
        if 1 <= cnt < 1.5:
            return '1'
        elif 1.5 <= cnt < 2.5:
            return '2'
        elif 2.5 <= cnt < 3.5:
            return '3'
        elif 3.5 <= cnt < 4.5:
            return '4'
        elif 4.5 <= cnt < 5.5:
            return '5'
        else:
            return '>5'

Так же разделим группы по квантилям. 

Так как наша метрика не всегда целочисленная, округлим дробные части до целых чисел в группах

In [107]:
df['camps_on_format_group'] = df['avg_camps_on_format'].apply(get_group_camps_on_format)

In [108]:
df['camps_on_format_group'].value_counts().reset_index().merge(round(df['camps_on_format_group'].value_counts(normalize=True) * 100, 2),
on='camps_on_format_group', how='left').merge(round(df.groupby('camps_on_format_group')['is_churned'].mean() * 100, 2).sort_values(ascending=False).reset_index(),
                             on='camps_on_format_group', how='left').sort_values(by='is_churned', ascending=False)

,camps_on_format_group,count,proportion,is_churned
0,1,451,27.32,88.25
1,2,411,24.89,82.73
4,4,148,8.96,77.03
3,3,206,12.48,74.76
5,5,88,5.33,64.77
2,>5,347,21.02,62.25


Видим похожую картину как и в случае с кампаниями

Самая лучшая стратегия - использовать множество кампаний на многих или одном формате